In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from _schon import ContinuousSIS
import horgg
import argparse
import json
import pickle
import ray
from time import time
from format_data import format_data
from hgcm import *
from collections import deque

In [10]:
def bfs(adj_dict, group_list, source_group, depth_limit):
    observed_group = {source_group}
    depth = 0
    queue = deque([(source_group,depth)])
    while queue and queue[0][1] < depth_limit:
        group_parent, depth = queue.popleft()
        for node in group_list[group_parent]:
            for group_child in adj_dict[node]:
                if group_child not in observed_group:
                    observed_group.add(group_child)
                    queue.append((group_child,depth +1))
    return observed_group

In [11]:
with open('./dat/coauth-DBLP_simplices.json','r') as file:
    unformatted_group_list = json.load(file)

In [12]:
#format data
data = format_data(unformatted_group_list)

In [40]:
adj_dict = data['adj_dict']
group_list = data['group_list']
source_group = np.random.randint(0,len(group_list))
depth_limit = 3
observed_groups = bfs(adj_dict, group_list, source_group, depth_limit)

In [41]:
subgroup_list = [group_list[g] for g in observed_groups]

In [52]:
sub_data = format_data(subgroup_list)
sub_data_to_save = {'group_list':sub_data['group_list'],
                    'adj_dict':sub_data['adj_dict'],
                    'edge_list':sub_data['edge_list']}

In [53]:
with open('./dat/coauth_subsample.json','w') as file:
    json.dump(sub_data_to_save,file)